# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hwange,-18.3693,26.5019,60.57,37,1,2.66,ZW,1627235475
1,1,Tamuín,21.9833,-98.7500,90.07,56,6,4.59,MX,1627235475
2,2,Dayong,25.0209,118.2898,87.48,84,85,4.27,CN,1627235475
3,3,Ushuaia,-54.8000,-68.3000,35.26,93,90,26.46,AR,1627235306
4,4,Saint-Louis,16.3333,-15.0000,101.84,32,56,11.14,SN,1627235475


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#heatmap of coordinates from cities_df
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Our dieal citi has a humidiy less and or equal to 50 and temperature between 70 and 90
best_weather_df = cities_df.loc[(cities_df['Humidity'] <=50) & (cities_df['Max Temp'] >=70) & (cities_df['Max Temp'] <90)]
best_weather_df.dropna()
best_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,Price,39.5994,-110.8107,86.72,38,1,0.00,US,1627235477
44,44,San Cristobal,7.7669,-72.2250,75.38,49,100,5.28,VE,1627235484
121,121,Sioux Lookout,50.1001,-91.9170,79.18,47,40,14.97,CA,1627235484
124,124,Saint-Augustin,51.2260,-58.6502,77.50,45,72,13.00,CA,1627235434
145,145,Sidi Ali,36.0998,0.4206,79.68,43,25,9.84,DZ,1627235506


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#New dataframe to store hotels
hotel_df = best_weather_df.loc[:,["City","Lat","Lng","Country"]]
hotel_df["Hotel"] = ""
hotel_df.head()

,City,Lat,Lng,Country,Hotel
12,Price,39.5994,-110.8107,US,
44,San Cristobal,7.7669,-72.2250,VE,
121,Sioux Lookout,50.1001,-91.9170,CA,
124,Saint-Augustin,51.2260,-58.6502,CA,
145,Sidi Ali,36.0998,0.4206,DZ,


In [7]:
#url and parameters for Google Places API
quary_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
#For loop to find hotels base on params
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    print(f"Processing {city_name}.")
    response = requests.get(quary_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel"] = results[0]['name']
    except (KeyError, IndexError):
        print(f"No hotels in {city_name}")
print("Search Completed")

Processing Price.
Closest hotel in Price is The Greenwell Inn.
Processing San Cristobal.
Closest hotel in San Cristobal is Lidotel Hotel Boutique San Cristobal.
Processing Sioux Lookout.
Closest hotel in Sioux Lookout is Moosehorn Lodge.
Processing Saint-Augustin.
No hotels in Saint-Augustin
Processing Sidi Ali.
No hotels in Sidi Ali
Processing Ananas.
No hotels in Ananas
Processing Mayskiy.
Closest hotel in Mayskiy is Restaurant and hotel complex Castle.
Processing Thunder Bay.
Closest hotel in Thunder Bay is Valhalla Inn Hotel.
Processing Iguape.
Closest hotel in Iguape is Hotéis em Ilha Comprida.
Processing Kaniama.
No hotels in Kaniama
Processing San Ramon.
Closest hotel in San Ramon is San Ramon Marriott.
Processing Makat.
Closest hotel in Makat is Gostinitsa Daulet.
Processing Krasnyy Yar.
Closest hotel in Krasnyy Yar is Hotel Alliance.
Processing Dosso.
Closest hotel in Dosso is Hotel GALAXY.
Processing Grivenskaya.
No hotels in Grivenskaya
Processing Marathon.
No hotels in Mara

In [10]:
hotel_df

,City,Lat,Lng,Country,Hotel
12,Price,39.5994,-110.8107,US,The Greenwell Inn
44,San Cristobal,7.7669,-72.2250,VE,Lidotel Hotel Boutique San Cristobal
121,Sioux Lookout,50.1001,-91.9170,CA,Moosehorn Lodge
124,Saint-Augustin,51.2260,-58.6502,CA,
145,Sidi Ali,36.0998,0.4206,DZ,
157,Ananas,19.2466,-71.9557,HT,
167,Mayskiy,47.6931,40.1025,RU,Restaurant and hotel complex Castle
168,Thunder Bay,48.4001,-89.3168,CA,Valhalla Inn Hotel
182,Iguape,-24.7081,-47.5553,BR,Hotéis em Ilha Comprida
194,Kaniama,-7.5667,24.1833,CD,


In [1]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))